**Problem definition:**

Optimisation problem
Your task is to schedule LPG delivery so that the overall cost of delivery is minimised, while making sure that:
each lorry starts from its home depot and return to any depot at the end of the journey,
you deliver to all customers who have less than 20% of gas in their tanks,
you fill the tanks of these customers to at least 50% of their capacity.

You are required to design and implement an optimisation scheme to address the LPG delivery scheduling problem.The optimisation scheme should use any combination of methods covered in class (e.g. Genetic Algorithms, Ant Colony Optimisation) as well as methods you have found during your independent study or which you came up with yourself. 
Your implementation must be in Python. You're allowed to use existing Python optimisation libraries or implementations if you need to, but you should aim to implement as much as possible from scratch yourself.You Must apply your implementation to the LPG delivery scheduling problem and critically evaluate the results, commenting on strengths and weaknesses of the approach you have used in terms of quality of the solution, running time etc.


**Methodology (all steps):**

In [5]:
pip install pulp

     |████████████████████████████████| 40.6MB 95kB/s 


In [6]:
import pandas as pd
import json 
import itertools
import numpy as np
from pulp import *

Here we have imported our dataset and created the output json file.

In [7]:
# dataset
optilandia_links=pd.read_csv("/content/SaO_Optilandia_resub_links.csv")
optilandia_locations=pd.read_csv("/content/SaO_Optilandia_resub_locations.csv")
optilandia_example_sol=pd.read_json("/content/SaO_Optilandia_resub_example_solution.json")

with open("/content/SaO_Optilandia_resub_depot_lorries.json") as json_data:
    data = json.load(json_data)

    
optilandia_depots=data

First we take the list of all supply nodes.
A list is a data structure in Python that is a mutable, or changeable, ordered sequence of elements. Each element or value that is inside of a list is called an item. Just as strings are defined as characters between quotes, lists are defined by having values between square brackets.


In [8]:
# a list of all the supply nodes
Depots=list(optilandia_depots.keys())

Then we have created a dictionary for the number of units of supply for each supply node
Dictionaries are Python's implementation of a data structure that is more generally known as an associative array. A dictionary consists of a collection of key-value pairs. Each key-value pair maps the key to its associated value.

In [ ]:
# Creates a dictionary for the number of units of supply for each supply node
supply={}
for i in range(len(Depots)):
    for j in optilandia_depots[Depots[i]]:
        supply.update({j['lorry_id']:j['capacity']})

Depots=list(supply.keys())

Here we have created a list of all demand nodes

In [ ]:
# Creates a list of all demand nodes
lpg_customers=optilandia_locations.dropna()
lpg_customers_nodes=list(optilandia_locations.dropna()['id'])

Here we have created a dictionary for the number of units of demand for each demand node.
We have used for loop to append our respective data.

In [9]:
# Creates a dictionary for the number of units of demand for each demand node
demand={}
lpg_customers_nodes=list(optilandia_locations.dropna()['id'])
lpg_customers_capacity=list(lpg_customers['capacity'])
lpg_customers_level=list(lpg_customers['level'])
# demand.update({lpg_customers_nodes:lpg_customers_level})
for a,b in zip(lpg_customers_nodes,lpg_customers_level):
    demand.update({a:b})

Here we have use greedy algorithms to find the shortest routes
Greedy is an algorithmic paradigm that builds up a solution piece by piece, always choosing the next piece that offers the most obvious and immediate benefit. So the problems where choosing locally optimal also leads to global solution are best fit for Greedy. For example consider the Fractional Knapsack Problem.
Dijkstra's algorithm is an algorithm for finding the shortest paths between nodes in a graph, which may represent, for example, road networks. Dijkstra's Algorithm stands out from the rest due to its ability to find the shortest path from one node to every other node within the same graph data structure. One algorithm for finding the shortest path from a starting node to a target node in a weighted graph is Dijkstra's algorithm. The algorithm creates a tree of shortest paths from the starting vertex, the source, to all other points in the graph. Dijkstra's algorithm is just BFS with a priority queue. Dijkstra's algorithm is used for graph searches. It is optimal, meaning it will find the single shortest path. It is uninformed, meaning it does not need to know the target node beforehand. In fact it finds the shortest path from every node to the node of origin.

In [10]:
# use greedy algorithms to find the shortest routes
depot_locations=optilandia_locations[optilandia_locations['is_depot']==True]
depot_116_shortest_route=[]
depot_124_shortest_route=[]
depot_373_shortest_route=[]
depot_523_shortest_route=[]
depot_passed=[]
depots_coords=[]
depot_ids=[]
lpg_customer_coords=[]
for a,b in zip(depot_locations['x'],depot_locations['y']):
    depots_coords.append(tuple((a,b)))
    depot_ids.append(depot_locations['id'])
for a,b in zip(lpg_customers['x'],lpg_customers['y']):
    lpg_customer_coords.append(tuple((a,b)))
depot_route=depots_coords+lpg_customer_coords
for r in depot_route:
    finish_1=49804.1667
    finish_2=49955.8333
    finish_3=49616.3889
    finish_4=49581.9444
    if finish_1 < r[0] and r[0] < finish_2:
        finish_1=r[1]
        depot_116_shortest_route.append(r)
    elif finish_2 < r[0]:
        finish_2=r[1]
        depot_124_shortest_route.append(r)
    elif finish_3 < r[0] and r[0] < finish_1 and r[0] < finish_2:
        finish_3=r[1]
        depot_373_shortest_route.append(r)
    elif finish_4 > r[0] or finish_4 < r[0] and r[0] < finish_1 and r[0] < finish_2 and r[0] < finish_3:
        finish_4=r[1]
        depot_523_shortest_route.append(r)
    else:
        depot_passed.append(r)
    

Here we have created costs functions

In [11]:
# costs function
depot_116_x=49804.1667
depot_116_y=6253.8889
depot_124_x=49955.8333
depot_124_y=5924.7222
depot_373_x=49616.3889
depot_373_y=5990.2778
depot_523_x=49581.9444
depot_523_y=6226.6667
depot_523_0=[] #5
depot_523_1=[] #5,
depot_523_2=[] #12,
depot_523_3=[] #12,
depot_523_4=[] #22,
depot_124_0=[] #5,
depot_124_1=[] #5,
depot_124_2=[] #5,
depot_124_3=[] #12,
depot_124_4=[] #12,
depot_124_5=[] #12,
depot_124_6=[] #22,
depot_124_7=[] #22,
depot_373_0=[] #5,
depot_373_1=[] #5,
depot_373_2=[] #5,
depot_373_3=[] #12,
depot_373_4=[] #12,
depot_373_5=[] #12,
depot_373_6=[] #12,
depot_373_7=[] #22,
depot_373_8=[] #22,
depot_373_9=[] #22,
depot_116_0=[] #5,
depot_116_1=[] #12
depot_116_df=[]
depot_124_df=[]
depot_373_df=[]
depot_523_df=[]

Here we have used a loop to calculate and append the optimal routes.
We have used this here linalg. Norm imported from numpy library.
This function is able to return one of eight different matrix norms, or one of an infinite number of vector norms (described below), depending on the value of the word parameter.
The norm is what is generally used to evaluate the error of a model. For instance it is used to calculate the error between the output of a neural network and what is expected (the actual label or value). You can think of the norm as the length of a vector. It is a function that maps a vector to a positive value.
numpy. linalg. norm is used to calculate the norm of a vector or a matrix. It take order=None as default, so just to calculate the Frobenius norm of (a-b) , this is to calculate the distance between a and b( using the upper Formula).
Solve a linear matrix equation, or system of linear scalar equations. ... Compute the (multiplicative) inverse of a matrix. linalg.pinv (a[, rcond, hermitian]) Compute the (Moore-Penrose) pseudo-inverse of a matrix.
The set of vectors whose 1-norm is a given constant forms the surface of a cross polytope of dimension equivalent to that of the norm minus 1. The Taxicab norm is also called the 1 norm. The distance derived from this norm is called the Manhattan distance or 1. Distance.
However it is possible to define something like that, in which case a vector whose norm is a hyperreal which is larger than all standard real numbers can be considered as having an infinite norm. Do note that this "norm" need not be compatible with the vector space structure, and a lot of modifications may be required.


In [ ]:
for i in range(len(depot_116_shortest_route)):
    depot_116_0.append((np.linalg.norm(np.array((depot_116_x,depot_116_y,0))-np.array((depot_116_shortest_route[i][0],depot_116_shortest_route[i][1],0))))*(1.00+5*1.50))
    depot_116_1.append((np.linalg.norm(np.array((depot_116_x,depot_116_y,0))- np.array((depot_116_shortest_route[i][0],depot_116_shortest_route[i][1],0))))*(1.50+12*1.00))
    depot_116_df.append(optilandia_locations[(optilandia_locations['x']==depot_116_shortest_route[i][0]) & (optilandia_locations['y'] == depot_116_shortest_route[i][1])] )
for i in range(len(depot_124_shortest_route)):
    depot_124_0.append((np.linalg.norm(np.array((depot_124_x,depot_124_y,0))-np.array((depot_124_shortest_route[i][0],depot_124_shortest_route[i][1],0))))*(1.00+5*1.50))
    depot_124_1.append((np.linalg.norm(np.array((depot_124_x,depot_124_y,0))-np.array((depot_124_shortest_route[i][0],depot_124_shortest_route[i][1],0))))*(1.00+5*1.50))
    depot_124_2.append((np.linalg.norm(np.array((depot_124_x,depot_124_y,0))-np.array((depot_124_shortest_route[i][0],depot_124_shortest_route[i][1],0))))*(1.00+5*1.50))
    depot_124_3.append((np.linalg.norm(np.array((depot_124_x,depot_124_y,0))- np.array((depot_124_shortest_route[i][0],depot_124_shortest_route[i][1],0))))*(1.50+12*1.00))
    depot_124_4.append((np.linalg.norm(np.array((depot_124_x,depot_124_y,0))- np.array((depot_124_shortest_route[i][0],depot_124_shortest_route[i][1],0))))*(1.50+12*1.00))
    depot_124_5.append((np.linalg.norm(np.array((depot_124_x,depot_124_y,0))- np.array((depot_124_shortest_route[i][0],depot_124_shortest_route[i][1],0))))*(1.50+12*1.00))
    depot_124_6.append((np.linalg.norm(np.array((depot_124_x,depot_124_y,0))- np.array((depot_124_shortest_route[i][0],depot_124_shortest_route[i][1],0))))*(2.00+22*0.50))
    depot_124_7.append((np.linalg.norm(np.array((depot_124_x,depot_124_y,0))- np.array((depot_124_shortest_route[i][0],depot_124_shortest_route[i][1],0))))*(2.00+22*0.50))
    depot_124_df.append(optilandia_locations[(optilandia_locations['x']==depot_124_shortest_route[i][0]) & (optilandia_locations['y'] == depot_124_shortest_route[i][1])] )
for i in range(len(depot_373_shortest_route)):
    depot_373_0.append((np.linalg.norm(np.array((depot_373_x,depot_373_y,0))-np.array((depot_373_shortest_route[i][0],depot_373_shortest_route[i][1],0))))*(1.00+5*1.50))
    depot_373_1.append((np.linalg.norm(np.array((depot_373_x,depot_373_y,0))-np.array((depot_373_shortest_route[i][0],depot_373_shortest_route[i][1],0))))*(1.00+5*1.50))
    depot_373_2.append((np.linalg.norm(np.array((depot_373_x,depot_373_y,0))-np.array((depot_373_shortest_route[i][0],depot_373_shortest_route[i][1],0))))*(1.00+5*1.50))
    depot_373_3.append((np.linalg.norm(np.array((depot_373_x,depot_373_y,0))- np.array((depot_373_shortest_route[i][0],depot_373_shortest_route[i][1],0))))*(1.50+12*1.00))
    depot_373_4.append((np.linalg.norm(np.array((depot_373_x,depot_373_y,0))- np.array((depot_373_shortest_route[i][0],depot_373_shortest_route[i][1],0))))*(1.50+12*1.00))
    depot_373_5.append((np.linalg.norm(np.array((depot_373_x,depot_373_y,0))- np.array((depot_373_shortest_route[i][0],depot_373_shortest_route[i][1],0))))*(1.50+12*1.00))
    depot_373_6.append((np.linalg.norm(np.array((depot_373_x,depot_373_y,0))- np.array((depot_373_shortest_route[i][0],depot_373_shortest_route[i][1],0))))*(1.50+12*0.50))
    depot_373_7.append((np.linalg.norm(np.array((depot_373_x,depot_373_y,0))- np.array((depot_373_shortest_route[i][0],depot_373_shortest_route[i][1],0))))*(2.00+22*0.50))
    depot_373_8.append((np.linalg.norm(np.array((depot_373_x,depot_373_y,0))- np.array((depot_373_shortest_route[i][0],depot_373_shortest_route[i][1],0))))*(2.00+22*0.50))
    depot_373_9.append((np.linalg.norm(np.array((depot_373_x,depot_373_y,0))- np.array((depot_373_shortest_route[i][0],depot_373_shortest_route[i][1],0))))*(2.00+22*0.50))
    depot_373_df.append(optilandia_locations[(optilandia_locations['x']==depot_373_shortest_route[i][0]) & (optilandia_locations['y'] == depot_373_shortest_route[i][1])] )
for i in range(len(depot_523_shortest_route)):
    depot_523_0.append((np.linalg.norm(np.array((depot_523_x,depot_523_y,0))-np.array((depot_523_shortest_route[i][0],depot_523_shortest_route[i][1],0))))*(1.00+5*1.50))
    depot_523_1.append((np.linalg.norm(np.array((depot_523_x,depot_523_y,0))-np.array((depot_523_shortest_route[i][0],depot_523_shortest_route[i][1],0))))*(1.00+5*1.50))
    depot_523_2.append((np.linalg.norm(np.array((depot_523_x,depot_523_y,0))-np.array((depot_523_shortest_route[i][0],depot_523_shortest_route[i][1],0))))*(1.50+12*1.00))
    depot_523_3.append((np.linalg.norm(np.array((depot_523_x,depot_523_y,0))- np.array((depot_523_shortest_route[i][0],depot_523_shortest_route[i][1],0))))*(1.50+12*1.00))
    depot_523_4.append((np.linalg.norm(np.array((depot_523_x,depot_523_y,0))- np.array((depot_523_shortest_route[i][0],depot_523_shortest_route[i][1],0))))*(2.00+22*0.50))
    depot_523_df.append(optilandia_locations[(optilandia_locations['x']==depot_523_shortest_route[i][0]) & (optilandia_locations['y'] == depot_523_shortest_route[i][1])] )


In [12]:
costs=[
    list(np.transpose(depot_116_0)),
    list(np.transpose(depot_116_1)),
    list(np.transpose(depot_124_0)),
    list(np.transpose(depot_124_1)),
    list(np.transpose(depot_124_2)),
    list(np.transpose(depot_124_3)),
    list(np.transpose(depot_124_4)),
    list(np.transpose(depot_124_5)),
    list(np.transpose(depot_124_6)),
    list(np.transpose(depot_124_7)),
    list(np.transpose(depot_373_0)),
    list(np.transpose(depot_373_1)),
    list(np.transpose(depot_373_2)),
    list(np.transpose(depot_373_3)),
    list(np.transpose(depot_373_4)),
    list(np.transpose(depot_373_5)),
    list(np.transpose(depot_373_6)),
    list(np.transpose(depot_373_7)),
    list(np.transpose(depot_373_8)),
    list(np.transpose(depot_373_9)),
    list(np.transpose(depot_523_0)),
    list(np.transpose(depot_523_1)),
    list(np.transpose(depot_523_2)),
    list(np.transpose(depot_523_3)),
    list(np.transpose(depot_523_4)),
]

Here The cost data is made into a dictionary

In [13]:
# The cost data is made into a dictionary
costs = makeDict([Depots,lpg_customers_nodes],costs,0)

Here we have created the 'prob' variable to contain the problem data
Here we have used LpProblem which is class from pulp library,
PuLP is an open source linear programming package for python. ... PuLP supports open source linear programming solvers such as CBC and GLPK, as well as commercial solvers such as Gurobi and IBM's CPLEX. The default solver is CBC, which comes packaged with PuLP upon installation.

This function creates a new LP Problem with the specified associated parameters
A linear combination of LpVariables. Can be initialised with the following:

    e = None: an empty Expression
    e = dict: gives an expression with the values being the coefficients of the keys (order of terms is undetermined)
    e = list or generator of 2-tuples: equivalent to dict.items()
    e = LpElement: an expression of length 1 with the coefficient 1
    e = other: the constant is initialised as e


In [14]:
# Creates the 'prob' variable to contain the problem data
prob = LpProblem("LPG Delivery Scheduling",LpMinimize)

/usr/local/lib/python3.7/dist-packages/pulp/pulp.py:1199: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Here we have created a list of tuples containing all the possible routes for transport

In [15]:
# Creates a list of tuples containing all the possible routes for transport
Routes = [(w,b) for w in Depots for b in lpg_customers_nodes]

Here a dictionary called 'Vars' is created to contain the referenced variables(the routes)
Using LpVariable.dicts() LpVariable(name, indexs, lowBound=None, upBound=None, cat='Continuous') name = The pre x to the name of each LP variable created. indexs = A list of strings of the keys to the dictionary of LP variables.


In [16]:
# A dictionary called 'Vars' is created to contain the referenced variables(the routes)
vars = LpVariable.dicts("Route",(Depots,lpg_customers_nodes),0,None,LpInteger)

Here the cost function is added to 'prob' first

In [17]:
# The cost function is added to 'prob' first
prob += lpSum([vars[w][b]*costs[w][b] for (w,b) in Routes]), "Sum_of_Transporting_Costs"

Here the supply maximum constraints are added to prob for each supply node (warehouse)

In [18]:
# The supply maximum constraints are added to prob for each supply node (warehouse)
for w in Depots:
    prob += lpSum([vars[w][b] for b in lpg_customers_nodes])<=supply[w], "Sum_of_Products_out_of_Depots_%s"%w


Here the demand minimum constraints are added to prob for each demand node (bar)

In [ ]:
# The demand minimum constraints are added to prob for each demand node (bar)
for b in lpg_customers_nodes:
    prob += lpSum([vars[w][b] for w in Depots])>=demand[b], "Sum_of_Products_into_Customer%s"%b

Here we have printed the prob variable with the solve function.

In [20]:
prob.solve()

-1

Here we have calculated the optimal solution

In [21]:
optimal_solution={}
optimal_solution_name=[]
optimal_solution_value=[]
for v in prob.variables():
    if v.varValue is not None and v.varValue > 0:
        optimal_solution_name.append(v.name)
        optimal_solution_value.append(v.varValue)
        optimal_solution.update({v.name:v.varValue})

In [22]:
optimal_route_name=[]
optimal_customer=[]
locations_=[]
results=[]
for i in range(len(optimal_solution_name)):
#     optimal_route.update({optimal_solution_name[i][6:11]:optimal_solution_name[i][12:16]})
        if optimal_solution_name[i][6:11]==optimal_solution_name[i][6:11]:
            locations_.append(optilandia_locations[optilandia_locations['id']==int(optimal_solution_name[i][12:16])])
            optimal_route_name.append(optimal_solution_name[i][6:11])
            optimal_customer.append([optimal_solution_name[i][12:16],optimal_solution_value[i]])

for i in range(len(locations_)):
    results.append([optimal_solution_name[i][6:11],[[optimal_solution_name[i][12:16],-(locations_[i]['capacity']-locations_[i]['level'])]]])
            

Here we have giving the column name to our generated dataframe

In [92]:
df=pd.DataFrame(results,columns=['lorry_id','loc'])

In [93]:
df

,lorry_id,loc
0,117_0,"[[215, [-0.6100000000000001]]]"
1,117_0,"[[279, [-1.23]]]"
2,117_0,"[[297, [-1.03]]]"
3,117_0,"[[357, [-0.83]]]"
4,117_0,"[[503, [-0.3700000000000001]]]"
...,...,...
120,522_2,"[[630, [-0.57]]]"
121,522_2,"[[64, [-0.20999999999999996]]]"
122,522_2,"[[7, [-0.29000000000000004]]]"
123,522_2,"[[71, [-0.84]]]"


Here we are doing the groupby of lorry ids and them sum of those ids

In [111]:
solution=df.groupby(["lorry_id"], as_index=False, sort=False).sum()

In [117]:
solution["lorry_id"] = solution["lorry_id"].str.replace("_","-")

Here we have printed the solution.

In [118]:
solution

,lorry_id,loc
0,117-0,"[[215, [-0.6100000000000001]], [279, [-1.23]],..."
1,117-1,"[[196, [-0.48]], [254, [-0.4099999999999999]],..."
2,125-0,"[[141, [-0.79]], [15, [-0.22999999999999998]],..."
3,125-1,"[[10, [-1.2]], [129, [-0.94]], [149, [-0.31999..."
4,125-2,"[[170, [-0.54]], [176, [-0.6599999999999999]],..."
5,372-0,"[[170, [-0.54]], [198, [-1.38]], [476, [-0.56]..."
6,372-1,"[[193, [-0.81]], [209, [-0.56]], [243, [-0.93]..."
7,522-0,"[[130, [-0.97]], [140, [-0.09000000000000002]]..."
8,522-1,"[[264, [-0.22999999999999998]], [327, [-1.88]]..."
9,522-2,"[[126, [-0.22999999999999998]], [144, [-0.76]]..."


In [119]:
# storing the data in JSON format
solution.to_json('/content/output.json', orient = 'split', compression = 'infer', index = 'true')

Here we have printed the example solution that is given by the professor.

In [27]:
optilandia_example_sol

,lorry_id,loc
0,522-0,"[[522, 4], [10, -1.2], [522, 0]]"
1,125-0,"[[125, 4], [595, -0.099999999999999], [469, -0..."
2,117-0,"[[117, 4], [117, 0]]"
3,372-0,"[[372, 4], [369, -0.459999999999999], [372, 0]]"
4,522-1,"[[522, 4], [627, -0.51], [522, 0]]"
5,125-1,"[[125, 11], [580, -0.169999999999999], [125, 0]]"
6,117-1,"[[117, 11], [117, 0]]"
7,372-1,"[[372, 11], [372, 0]]"
8,522-2,"[[522, 24], [522, 0]]"
9,125-2,"[[125, 11], [125, 0]]"


**Experiments & discussion:**

Basically we have used greedy algorithm to find the shortest routes, Greedy is an algorithmic paradigm that builds up a solution piece by piece, always choosing the next piece that offers the most obvious and immediate benefit. So the problems where choosing locally optimal also leads to global solution are best fit for Greedy. For example consider the Fractional Knapsack Problem
we have tried Genetic Algorithms & Ant Colony Optimisation algorithms also as per suggested, But those algorithms took too much time to learn. At the end we discuss and decide to develop our own model with the help of greedy algorithms.
Route optimization is the process of determining the shortest possible routes to reach a location. This methodology has gained popularity in the transport and logistics industry. Since it reduces the time spent traveling and at the same time reduces the incurred cost in the process.

Before a company or an organization can employ this strategy, it has to first be able to document all of its routes of business after which experts can then use the provided data to predict the best possible routes.

In the course of determining the best possible routes, there is usually a simulation of different scenarios. And ever since route optimization has gone digital with lots of software available today, the prediction of these routes is now done using the route optimization algorithm.

To solve an optimization problem, begin by drawing a picture and introducing variables. Find an equation relating the variables. Find a function of one variable to describe the quantity that is to be minimized or maximized. Look for critical points to locate local extrema.

To better appreciate and understand what route optimization algorithm is expected to solve, knowledge of vehicle routing problem is important.

To solve optimization problem there are default steps,

    Import the required libraries.
    Declare the solver.
    Create the variables.
    Define the constraints.
    Define the objective function.
    Invoke the solver and display the results.


**Conclusion and References:**

As a conclusion we have found out the better root in a very fast and easy manner using greedy algorithms, we have tried Genetic Algorithms & Ant Colony Optimisation algorithms also as per suggested, But those algorithms took too much time to learn. At the end we discuss and decide to develop our own model with the help of greedy algorithms.


1. https://brilliant.org/wiki/greedy-algorithm/
2. https://towardsdatascience.com/improving-operations-with-route-optimization-4b8a3701ca39
3. https://towardsdatascience.com/travel-time-optimization-with-machine-learning-and-genetic-algorithm-71b40a3a4c2
4. https://developers.google.com/optimization/routing/vrp
5. https://gsmtasks.com/route-optimization-algorithm-and-big-data/